### Diversification and Top Performers by Sector

1. **Notebook Objective:**
	- This notebook will group stocks by sector
	- It will identify the top *n* stocks in each sector based on user-defined criteria (n is defined in `config.py`)
	- It will compute the correlation matrix of returns across selected stocks
		- Notation: Let $R$ be a matrix of returns, then the correlation matrix $\rho$ is given by  
		  $\rho_{i,j} = \frac{\text{Cov}(R_i, R_j)}{\sigma_{R_i} \cdot \sigma_{R_j}}$
	- It will identify the most diversifiable stocks (those with the lowest correlations)
	- Finally, it will export potential portfolios—those with high Sharpe Ratios and strong performance metrics—to `config.py` for use in later analysis

2. **Bugs**
    - Need to fix function get_corr_pairs in utils.finance_utils.py to allow user to match date with program


In [1]:
import time
import os 
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
print(f'Current Working Directory: {os.getcwd()}')

from utils.finance_utils import *
from utils.helpers import separate_corr_pairs
from utils.config import TOP_N_STOCKS, PROGRAM_START_DATE, PROGRAM_END_DATE

Current Working Directory: /Users/blakeuribe/Desktop/portfolio_py/notebooks

---------------------------------
finance_utils.py successfully loaded, updated on 07/12/2025 2:57
---------------------------------



---------------------------------
helpers.py successfully loaded, updated last Feb. 04 2025
---------------------------------


Updated on 06/12/2025 3:11 2025-07-12


Sector Valuation: Group by sectors, and find best stock in each sector

In [2]:
start_time = time.time() # measure run time

filtered_valuation_df = pd.read_csv(f'../data/clean/filtered_valuation_df.csv')

top_stocks_on_profit = get_top_n_by_sector(filtered_valuation_df, filter_var='profitMargins', top_n=3)
top_stocks_on_sharpe = get_top_n_by_sector(filtered_valuation_df, filter_var='Sharpe_ratios', top_n=3)

In [3]:
top_stocks_on_profit

,Tickers,Sharpe_ratios,Sector,title,marketCap,trailingPE,forwardPE,profitMargins,trailingEps,quickRatio,earningsQuarterlyGrowth
0,KGC,2.309481,Basic Materials,KINROSS GOLD CORP,1.911453e+10,15.867347,17.670454,0.21740,0.98,1.146,2.439
1,EBAY,1.574066,Consumer Cyclical,EBAY INC,3.553568e+10,18.536058,14.772032,0.19783,4.16,1.072,0.148
2,FTDR,1.376674,Consumer Cyclical,"Frontdoor, Inc.",4.268712e+09,18.895765,18.474522,0.12586,3.07,1.249,0.088
3,ATGE,1.497722,Consumer Defensive,Adtalem Global Education Inc.,4.239145e+09,19.964468,17.931610,0.13341,5.91,0.707,0.652
4,LRN,1.483840,Consumer Defensive,"Stride, Inc.",5.853449e+09,21.113031,18.524794,0.13098,6.37,5.281,1.147
5,MGY,1.443851,Energy,Magnolia Oil & Gas Corp,4.584683e+09,11.792079,12.215385,0.28503,2.02,1.329,0.210
6,MAIN,1.372579,Financial Services,Main Street Capital CORP,5.606181e+09,10.672881,16.313473,0.94611,5.90,2.641,0.083
7,FINV,1.867060,Financial Services,FinVolution Group,2.686189e+09,7.625900,7.210885,0.19415,1.39,3.585,0.414
8,ADMA,1.505229,Healthcare,"ADMA BIOLOGICS, INC.",4.418967e+09,22.035715,25.708332,0.45011,0.84,3.197,0.511
9,ABT,1.292091,Healthcare,ABBOTT LABORATORIES,2.296937e+11,17.145456,25.585274,0.31886,7.70,1.090,0.082


Diversifaction: Find negative correlated stocks

In [4]:
top_profit_corr_pairs = get_corr_pairs_of_stocks(top_stocks_on_profit['Tickers'].tolist(), start_date=PROGRAM_START_DATE, end_date=PROGRAM_END_DATE)
top_sharpe_corr_pairs = get_corr_pairs_of_stocks(top_stocks_on_sharpe['Tickers'].tolist(), start_date=PROGRAM_START_DATE, end_date=PROGRAM_END_DATE)

mpt_stocks_for_sharpe = separate_corr_pairs(top_sharpe_corr_pairs.index.to_list(), top_n=TOP_N_STOCKS)
mpt_stocks_for_profit = separate_corr_pairs(top_profit_corr_pairs.index.to_list(), top_n=TOP_N_STOCKS)

print(f'Top Stocks based on Sharpe: {mpt_stocks_for_sharpe}')
print(f'Top Stocks based on Profit Margin: {mpt_stocks_for_profit}')

[*********************100%***********************]  14 of 14 completed
[*********************100%***********************]  14 of 14 completed

Top Stocks based on Sharpe: ['LRN' 'MGY' 'KGC' 'ABT' 'ATGE' 'OMAB' 'FINV' 'IDCC']
Top Stocks based on Profit Margin: ['LRN' 'MGY' 'KGC' 'ABT' 'ATGE' 'OMAB' 'FINV' 'IDCC']


In [5]:
top_sharpe_corr_pairs

,Correlation
LRN_MGY,-0.596512
KGC_MGY,-0.589646
ABT_MGY,-0.570614
ATGE_MGY,-0.502688
MGY_OMAB,-0.499450
...,...
ABT_LRN,0.902090
ATGE_OMAB,0.911696
KGC_OMAB,0.923274
EXEL_IDCC,0.935067


Seperate low corr pairs

In [10]:
with open('../utils/config.py', 'a') as f:
    f.write('\n# New Entry from portfolio_selection.ipynb')
    f.write(f'\nmpt_stocks_for_sharpe = {(mpt_stocks_for_sharpe).tolist()}\n')
    f.write(f'mpt_stocks_for_profit = {(mpt_stocks_for_profit).tolist()}\n')
    # f.write(f'my_pairs = {(my_pairs).tolist()}\n')

In [7]:
filtered_valuation_df[filtered_valuation_df['Tickers'].isin(mpt_stocks_for_sharpe)].reset_index(drop=True)

,Tickers,Sharpe_ratios,Sector,title,marketCap,trailingPE,forwardPE,profitMargins,trailingEps,quickRatio,earningsQuarterlyGrowth
0,ABT,1.292091,Healthcare,ABBOTT LABORATORIES,2.296937e+11,17.145456,25.585274,0.31886,7.70,1.090,0.082
1,ATGE,1.497722,Consumer Defensive,Adtalem Global Education Inc.,4.239145e+09,19.964468,17.931610,0.13341,5.91,0.707,0.652
2,FINV,1.867060,Financial Services,FinVolution Group,2.686189e+09,7.625900,7.210885,0.19415,1.39,3.585,0.414
3,IDCC,1.878982,Technology,"InterDigital, Inc.",5.842761e+09,17.821993,46.639750,0.48139,12.64,1.710,0.416
4,KGC,2.309481,Basic Materials,KINROSS GOLD CORP,1.911453e+10,15.867347,17.670454,0.21740,0.98,1.146,2.439
5,LRN,1.483840,Consumer Defensive,"Stride, Inc.",5.853449e+09,21.113031,18.524794,0.13098,6.37,5.281,1.147
6,MGY,1.443851,Energy,Magnolia Oil & Gas Corp,4.584683e+09,11.792079,12.215385,0.28503,2.02,1.329,0.210
7,OMAB,1.456871,Industrials,Central North Airport Group,5.457172e+09,19.814360,19.506895,0.34505,5.71,1.523,0.193


In [8]:
filtered_valuation_df[filtered_valuation_df['Tickers'].isin(mpt_stocks_for_profit)].reset_index(drop=True)

,Tickers,Sharpe_ratios,Sector,title,marketCap,trailingPE,forwardPE,profitMargins,trailingEps,quickRatio,earningsQuarterlyGrowth
0,ABT,1.292091,Healthcare,ABBOTT LABORATORIES,2.296937e+11,17.145456,25.585274,0.31886,7.70,1.090,0.082
1,ATGE,1.497722,Consumer Defensive,Adtalem Global Education Inc.,4.239145e+09,19.964468,17.931610,0.13341,5.91,0.707,0.652
2,FINV,1.867060,Financial Services,FinVolution Group,2.686189e+09,7.625900,7.210885,0.19415,1.39,3.585,0.414
3,IDCC,1.878982,Technology,"InterDigital, Inc.",5.842761e+09,17.821993,46.639750,0.48139,12.64,1.710,0.416
4,KGC,2.309481,Basic Materials,KINROSS GOLD CORP,1.911453e+10,15.867347,17.670454,0.21740,0.98,1.146,2.439
5,LRN,1.483840,Consumer Defensive,"Stride, Inc.",5.853449e+09,21.113031,18.524794,0.13098,6.37,5.281,1.147
6,MGY,1.443851,Energy,Magnolia Oil & Gas Corp,4.584683e+09,11.792079,12.215385,0.28503,2.02,1.329,0.210
7,OMAB,1.456871,Industrials,Central North Airport Group,5.457172e+09,19.814360,19.506895,0.34505,5.71,1.523,0.193
